###### CLV:
In marketing, the CLV is one of the key metrics to have and monitor. The CLV measures customers' total worth to the business over the course of their lifetime relationship with the company. This metric is especially important to keep track of for acquiring new customers. 

It is generally more expensive to acquire new customers than to keep existing customers, so knowing the lifetime value and the costs associated with acquiring new customers is essential in order to build marketing strategies with a positive ROI. For example, if the average CLV of your customer is 100 and it only costs 10 to acquire a new customer, then your business will be generating more revenue as you acquire new customers.

However, if it costs 150 to acquire a new customer and the average CLV of your customer is still 100, then you will be losing money for each acquisition. Simply put, if your marketing spend for new customer acquisition exceeds the CLV, you will be losing money for each acquisition, and it is better to just work with the existing customers.


There are multiple ways to calculate CLV:
- One way is to find the customer's average purchase amount, purchase frequency, and lifetime span and do a simple calculation to get the CLV. e.g. The final CLV amount is calculated by multiplying 500, the average value per month, by 12 months and the lifetime span of 20 years

- CLV can also be estimated through building predictive models

we are going to learn how to build a **regression model** that predicts customers' 3-month CLV

Evaluating regression models:
Four commonly used methodologies to evaluate regression:
- models—mean squared error (MSE)
- median absolute error (MAE)
- R2
- predicted versus actual scatter plot

###### Case Study 7: Using a linear regression model to predict CLV (3 month customer value) for a online retailer

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_excel('Online Retail.xlsx', sheet_name='Online Retail')

In [2]:
# Data clean
    # Handling negative quantity:
df = df.loc[df['Quantity'] > 0]
    # Dropping NaN records
df = df[pd.notnull(df['CustomerID'])]
    # Handling incomplete data
df = df.loc[df['InvoiceDate'] < '2011-12-01']
    # Total sales value
df['Sales'] = df['Quantity'] * df['UnitPrice']

In [3]:
# let's summarize this data for each order
orders_df = df.groupby(['CustomerID', 'InvoiceNo']).agg({
            'Sales': sum,
            'InvoiceDate': max})
orders_df

Sales         InvoiceDate
CustomerID InvoiceNo                              
12346.0    541431     77183.60 2011-01-18 10:01:00
12347.0    537626       711.79 2010-12-07 14:57:00
           542237       475.39 2011-01-26 14:30:00
           549222       636.25 2011-04-07 10:43:00
           556201       382.52 2011-06-09 13:01:00
           562032       584.91 2011-08-02 08:48:00
           573511      1294.32 2011-10-31 12:25:00
12348.0    539318       892.80 2010-12-16 19:09:00
           541998       227.44 2011-01-25 10:42:00
           548955       367.00 2011-04-05 10:47:00
           568172       310.00 2011-09-25 13:13:00
12349.0    577609      1757.55 2011-11-21 09:51:00
12350.0    543037       334.40 2011-02-02 16:01:00
12352.0    544156       296.50 2011-02-16 12:33:00
           545323       144.35 2011-03-01 14:57:00
           545332       840.30 2011-03-01 15:52:00
           546869       120.33 2011-03-17 16:00:00
           547390       160.33 2011-03-22 16:08:00
           567505       366.25 2011-09-20 14:34:00
           568699       266.25 2011-09-28 14:58:00
           574275       311.73 2011-11-03 14:37:00
12353.0    553900        89.00 2011-05-19 17:47:00
12354.0    550911      1079.40 2011-04-21 13:11:00
12355.0    552449       459.40 2011-05-09 13:49:00
12356.0    541430      2271.62 2011-01-18 09:50:00
           549435       481.46 2011-04-08 12:33:00
           576895        58.35 2011-11-17 08:40:00
12357.0    574740      6207.67 2011-11-06 16:07:00
12358.0    559707       484.86 2011-07-12 10:04:00
12359.0    540946       547.50 2011-01-12 12:43:00
...                        ...                 ...
18272.0    559813       753.66 2011-07-12 15:42:00
           563680       372.25 2011-08-18 12:06:00
           572617       604.25 2011-10-25 11:52:00
18273.0    547843        51.00 2011-03-27 11:22:00
           565480       102.00 2011-09-05 11:27:00
18274.0    575485       175.92 2011-11-09 17:03:00
18276.0    572990       335.86 2011-10-27 10:54:00
18277.0    570859       110.38 2011-10-12 15:22:00
18278.0    568492       173.90 2011-09-27 11:58:00
18280.0    545712       180.60 2011-03-07 09:52:00
18281.0    556464        80.82 2011-06-12 10:53:00
18282.0    562525       100.21 2011-08-05 13:35:00
18283.0    540350       108.45 2011-01-06 14:14:00
           541854       106.55 2011-01-23 14:58:00
           545079       102.90 2011-02-28 10:30:00
           550957       117.68 2011-04-21 16:37:00
           554157        99.47 2011-05-23 11:33:00
           556731       103.72 2011-06-14 11:41:00
           557956       203.81 2011-06-23 19:20:00
           560025       140.69 2011-07-14 13:00:00
           560032         2.50 2011-07-14 13:20:00
           565579       134.90 2011-09-05 12:35:00
           573093       114.65 2011-10-27 14:38:00
           575668       112.35 2011-11-10 14:59:00
           575675         1.95 2011-11-10 15:07:00
           578262       313.65 2011-11-23 13:27:00
           579673       223.61 2011-11-30 12:59:00
18287.0    554065       765.28 2011-05-22 10:39:00
           570715      1001.32 2011-10-12 10:23:00
           573167        70.68 2011-10-28 09:29:00

[17758 rows x 2 columns]

In [4]:
# In order to calculate the CLV, we need to know the frequency, recency, and total amount of purchases by each customer

def groupby_mean(x): # computes the average for each group and the second function
    return x.mean()

def groupby_count(x): # counts the number of records in each group
    return x.count()

def purchase_duration(x):
    return (x.max() - x.min()).days # counts the number of days between the first and last invoice dates in each group

def avg_frequency(x):
    return (x.max() - x.min()).days/x.count() # calculates the average number of days between orders


groupby_mean.__name__ = 'avg'
groupby_count.__name__ = 'count'
purchase_duration.__name__ = 'purchase_duration'
avg_frequency.__name__ = 'purchase_frequency'

summary_df = orders_df.reset_index().groupby('CustomerID').agg({
    'Sales': [min, max, sum, groupby_mean, groupby_count],
    'InvoiceDate': [min, max, purchase_duration, avg_frequency]
})


In [5]:
summary_df

Sales                                          \
                 min       max       sum           avg count   
CustomerID                                                     
12346.0     77183.60  77183.60  77183.60  77183.600000   1.0   
12347.0       382.52   1294.32   4085.18    680.863333   6.0   
12348.0       227.44    892.80   1797.24    449.310000   4.0   
12349.0      1757.55   1757.55   1757.55   1757.550000   1.0   
12350.0       334.40    334.40    334.40    334.400000   1.0   
12352.0       120.33    840.30   2506.04    313.255000   8.0   
12353.0        89.00     89.00     89.00     89.000000   1.0   
12354.0      1079.40   1079.40   1079.40   1079.400000   1.0   
12355.0       459.40    459.40    459.40    459.400000   1.0   
12356.0        58.35   2271.62   2811.43    937.143333   3.0   
12357.0      6207.67   6207.67   6207.67   6207.670000   1.0   
12358.0       484.86    484.86    484.86    484.860000   1.0   
12359.0       547.50   2876.85   6372.58   1593.145000   4.0   
12360.0       534.70   1083.58   2662.06    887.353333   3.0   
12361.0       189.90    189.90    189.90    189.900000   1.0   
12362.0       303.76    829.99   4697.19    521.910000   9.0   
12363.0       252.90    299.10    552.00    276.000000   2.0   
12364.0        79.80    623.92   1002.78    334.260000   3.0   
12365.0       320.69    320.69    641.38    320.690000   2.0   
12370.0       277.20   1590.82   3545.69    886.422500   4.0   
12371.0       360.00   1527.96   1887.96    943.980000   2.0   
12372.0       383.12    515.70   1298.04    432.680000   3.0   
12373.0       364.60    364.60    364.60    364.600000   1.0   
12374.0       742.93    742.93    742.93    742.930000   1.0   
12375.0       227.20    230.30    457.50    228.750000   2.0   
12377.0       626.60   1001.52   1628.12    814.060000   2.0   
12378.0      4008.62   4008.62   4008.62   4008.620000   1.0   
12379.0       392.40    459.84    852.24    426.120000   2.0   
12380.0       450.86   1040.39   2724.81    681.202500   4.0   
12381.0         4.50   1268.74   1698.30    424.575000   4.0   
...              ...       ...       ...           ...   ...   
18242.0       312.92   1225.49   2232.49    558.122500   4.0   
18245.0        35.60    439.18   1672.81    278.801667   6.0   
18246.0       232.90    363.20    596.10    298.050000   2.0   
18248.0       307.56    496.46    804.02    402.010000   2.0   
18249.0        95.34     95.34     95.34     95.340000   1.0   
18250.0       349.27    349.27    349.27    349.270000   1.0   
18251.0      4314.72   4314.72   4314.72   4314.720000   1.0   
18252.0        69.90    456.77    526.67    263.335000   2.0   
18255.0       103.30    103.30    103.30    103.300000   1.0   
18257.0        14.85    627.27   2337.63    292.203750   8.0   
18259.0       376.30   1070.40   2338.60    779.533333   3.0   
18260.0       230.70    557.07   2643.20    440.533333   6.0   
18261.0        99.44    224.80    324.24    162.120000   2.0   
18262.0       149.48    149.48    149.48    149.480000   1.0   
18263.0       307.60    503.80   1213.16    404.386667   3.0   
18265.0       312.96    488.55    801.51    400.755000   2.0   
18268.0        25.50     25.50     25.50     25.500000   1.0   
18269.0       168.60    168.60    168.60    168.600000   1.0   
18270.0       111.95    171.20    283.15    141.575000   2.0   
18272.0       340.72    753.66   2710.70    542.140000   5.0   
18273.0        51.00    102.00    153.00     76.500000   2.0   
18274.0       175.92    175.92    175.92    175.920000   1.0   
18276.0       335.86    335.86    335.86    335.860000   1.0   
18277.0       110.38    110.38    110.38    110.380000   1.0   
18278.0       173.90    173.90    173.90    173.900000   1.0   
18280.0       180.60    180.60    180.60    180.600000   1.0   
18281.0        80.82     80.82     80.82     80.820000   1.0   
18282.0       100.21    100.21    100.21    100.210000   1.0   
18283.0         1.95    313.65   1886.88    125

In [6]:
for col in summary_df.columns:
    print(col)

('Sales', 'min')
('Sales', 'max')
('Sales', 'sum')
('Sales', 'avg')
('Sales', 'count')
('InvoiceDate', 'min')
('InvoiceDate', 'max')
('InvoiceDate', 'purchase_duration')
('InvoiceDate', 'purchase_frequency')


In [7]:
summary_df.columns = ['_'.join(col).lower() for col in summary_df.columns]
summary_df.columns

Index(['sales_min', 'sales_max', 'sales_sum', 'sales_avg', 'sales_count',
       'invoicedate_min', 'invoicedate_max', 'invoicedate_purchase_duration',
       'invoicedate_purchase_frequency'],
      dtype='object')

In [8]:
# Let's take a closer look at the distributions of the number of purchases that the repeat customers have made
summary_df = summary_df.loc[summary_df['invoicedate_purchase_duration'] > 0]

ax = summary_df.groupby('sales_count').count()['sales_avg'][:20].plot(
    kind='bar', 
    color='skyblue',
    figsize=(12,7), 
    grid=True
)

ax.set_ylabel('count')

plt.show()

# As you can see from this plot, the majority of customers have made 10 or less purchases historically

NameError: name 'plt' is not defined

In [ ]:
# Let's take a look at the average number of days between purchases for these repeat customers
ax = summary_df['invoicedate_purchase_frequency'].hist(
    bins=20,
    color='skyblue',
    rwidth=0.7,
    figsize=(12,7)
)

ax.set_xlabel('avg. number of days between purchases')
ax.set_ylabel('count')

plt.show()

# the majority of repeat customers made purchases every 20 to 50 days

Now, we are going to build a model that predicts the 3 month customer value using the pandas and scikit-learn packages in Python. We are going to first slice the data into chunks of 3 months and take **the last 3 months' data as the target** for predictions and the rest as the features

In [ ]:
# Data preparation:
clv_freq = '3M'

data_df = orders_df.reset_index().groupby([
    'CustomerID',
    pd.Grouper(key='InvoiceDate', freq=clv_freq)
]).agg({
    'Sales': [sum, groupby_mean, groupby_count],
})

data_df.columns = ['_'.join(col).lower() for col in data_df.columns]
data_df = data_df.reset_index()
data_df

Since we want to predict the 3 month customer value, we are **breaking down the data into chunks of 3 months for each customer**. As you can see in the groupby function, we group the previously built DataFrame orders_df by CustomerID and a custom Grouper, which groups InvoiceDate by every 3 months. Then, for each group of 3 month time windows, we sum up all of the sales to get the total purchase amount, take the average of purchase amount and the total number of purchases for the given period for each customer

In [ ]:
# let's encode the InvoiceDate column values so that they are easier to read
date_month_map = {
    str(x)[:10]: 'M_%s' % (i+1) for i, x in enumerate(
        sorted(data_df.reset_index()['InvoiceDate'].unique(), reverse=True)
    )
}

data_df['M'] = data_df['InvoiceDate'].apply(lambda x: date_month_map[str(x)[:10]])
data_df

# As you can see from this code, we are encoding date values into M_1, M_2, M_3, and so forth

As briefly mentioned before, we are going to use the last 3 months as the target variable and the rest as the features, meaning we are going to train a machine learning model that predicts the last 3 months' customer value with the rest of the data. In order to train such a model, we need to **transform this data into tabular data**, where the rows represent the individual customers and the columns represent each feature

In [ ]:
features_df = pd.pivot_table(
    data_df.loc[data_df['M'] != 'M_1'], 
    values=['sales_sum', 'sales_avg', 'sales_count'], 
    columns='M', 
    index='CustomerID'
)

features_df.columns = ['_'.join(col) for col in features_df.columns]
features_df # don't need reset_index() this time
# Recall how we did this in SQL

In [ ]:
# encode these NaN values with 0.0
features_df = features_df.fillna(0)
features_df

In [ ]:
# let's build the target variables
response_df = data_df.loc[
    data_df['M'] == 'M_1',
    ['CustomerID', 'sales_sum']
]

response_df.columns = ['CustomerID', 'CLV_'+clv_freq]

response_df

#we are taking the last 3 month period, the M_1 group, as the target variable. 
#The target column will be sales_sum, as we want to predict the next 3 month customer value, 
# which is the total purchase amount that a given customer is likely to make in the next 3 months

In [ ]:
# combine features and response data together to build a model
sample_set_df = features_df.merge(
    response_df, 
    left_index=True, # Use the index from the left DataFrame as the join key(s)
    right_on='CustomerID',
    how='left' # By having the how='left' flag, 
               # we take all records in the features data, 
                # even if there is no corresponding data in the response data
)

sample_set_df = sample_set_df.fillna(0) #  This is a case where the given customer did not make any purchases in the last 3 months, 
                                        #  so we encode them as zero
sample_set_df

In [ ]:
# Linear regression:
from sklearn.model_selection import train_test_split

target_var = 'CLV_'+clv_freq
all_features = [x for x in sample_set_df.columns if x not in ['CustomerID', target_var]]

x_train, x_test, y_train, y_test = train_test_split(
    sample_set_df[all_features], 
    sample_set_df[target_var], 
    test_size=0.3
)

In [ ]:
# train a model
from sklearn.linear_model import LinearRegression

reg_fit = LinearRegression()
reg_fit.fit(x_train, y_train)

In [ ]:
#  use the intercept_ attribute of the LinearRegression object
reg_fit.intercept_

In [ ]:
# find the fitted linear regression model's coefficients
reg_fit.coef_

In [ ]:
coef=pd.DataFrame(list(zip(all_features,reg_fit.coef_)))
coef.columns= ['features','coef']
coef

As you can see from this coefficient output, you can easily find which features have negative correlation with the target and which features have positive correlation with the target. For example, the previous 3 month period's average purchase amount, sales_avg_M_2, has negative impacts on the next 3 month customer value. This means that the higher the previous 3 month period's purchase amount is, the lower the next 3 month purchase amount will be.

Using the 3 month customer value prediction output, you can custom-tailor your marketing strategies in different ways. Since you know the expected revenue or purchase amount from individual customers for the next 3 months, you can set a better informed budget for your marketing campaign. It should be set high enough to reach your target customers, but low enough to be below the expected 3 month customer value, so that you can have a positive ROI marketing campaign. 

On the other hand, you can also use these 3 month customer value prediction output values to specifically target these high-value customers for the next 3 months. This can help you to create marketing campaigns with a higher ROI, as those high-value customers, predicted by this model, are likely to bring in more revenue than the others